In [3]:
pip install transformers scikit-learn

Note: you may need to restart the kernel to use updated packages.


## 这个版本demo1 （可以正确提交但是排名倒数第二）
读取并解析JSON格式的数据。

使用简单的文本处理方法来学习support_set中的正样本特征。

在query_set上应用这些特征以区分正负样本。

输出一个CSV文件，包含task_id、record_id（这里简化处理，使用query_set中的索引作为record_id）和label。


In [11]:
# 这个版本
import json
import csv

def read_data(file_path):
    """读取并解析JSON格式的数据"""
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            yield json.loads(line)

def extract_keywords(support_set):
    """从support_set中提取关键词作为简化的正样本特征"""
    keywords = set()
    for item in support_set:
        # 假设简单地将每个支持集条目的文本拆分为词，并全部加入关键词集
        keywords.update(item['text'].split())
    return keywords

def predict(query_set, keywords):
    """根据关键词预测query_set中的每个样本是否为正样本"""
    predictions = []
    for idx, item in enumerate(query_set):
        # 简化判断：如果文本中包含任一关键词，则预测为正样本
        text_keywords = set(item['text'].split())
        label = 1 if keywords & text_keywords else 0
        predictions.append((idx, label))
    return predictions

def write_predictions_to_csv(predictions, task_id, output_file):
    """将预测结果写入CSV文件"""
    with open(output_file, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for record_id, label in predictions:
            writer.writerow([task_id, record_id, label])

def process_file(input_file_path, output_file_path):
    """处理整个文件，并输出预测结果到CSV"""
    data = read_data(input_file_path)
    with open(output_file_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['task_id', 'record_id', 'label'])  # 写入表头
    for task in data:
        keywords = extract_keywords(task['support_set'])
        predictions = predict(task['query_set'], keywords)
        write_predictions_to_csv(predictions, task['task_id'], output_file_path)

# 调用上述函数
input_file_path = './data/train_data.txt'  # 修改为实际文件路径
output_file_path = './submission.csv'  # 您希望生成的提交文件路径
process_file(input_file_path, output_file_path)


In [8]:
import csv
import json
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# 初始化BERT模型和分词器
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

def encode_texts(texts):
    """对一组文本进行编码"""
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    # 取最后一层的隐藏状态的平均作为特征表示
    embeddings = outputs.last_hidden_state.mean(1)
    return embeddings

def read_data(file_path):
    """从文件中读取并解析JSON数据"""
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            yield json.loads(line)

def find_positive_samples(task, threshold=0.5):
    """识别query_set中的正样本"""
    support_texts = [item['text'] for item in task['support_set']]
    query_texts = [item['text'] for item in task['query_set']]
    support_embeddings = encode_texts(support_texts)
    query_embeddings = encode_texts(query_texts)
    similarities = cosine_similarity(query_embeddings, support_embeddings)
    max_similarities = similarities.max(axis=1)
    labels = [1 if sim >= threshold else 0 for sim in max_similarities]
    return labels

def process_tasks_and_write_output(input_file_path, output_file_path):
    """处理所有任务并将结果写入CSV文件"""
    with open(output_file_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['task_id', 'record_id', 'label'])  # 写入表头
        
        for task in read_data(input_file_path):
            labels = find_positive_samples(task)
            for record_id, label in enumerate(labels):
                writer.writerow([task['task_id'], record_id, label])

# 设置文件路径
# input_file_path = 'path/to/your/train_data.txt'
# output_file_path = 'path/to/your/submission.csv'
input_file_path = './data/train_data.txt'  # 修改为实际文件路径
output_file_path = './submission.csv'  # 您希望生成的提交文件路径

# 处理数据并输出结果
process_tasks_and_write_output(input_file_path, output_file_path)


KeyboardInterrupt: 

In [10]:
import csv
import json
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# 初始化BERT模型和分词器
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModel.from_pretrained('bert-base-uncased')

def encode_texts(texts):
    """对一组文本进行编码"""
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    # 取最后一层的隐藏状态的平均作为特征表示
    embeddings = outputs.last_hidden_state.mean(1)
    return embeddings

def read_data(file_path):
    """从文件中读取并解析JSON数据"""
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            yield json.loads(line)

def find_positive_samples(task, threshold=0.5):
    """识别query_set中的正样本"""
    support_texts = [item['text'] for item in task['support_set']]
    query_texts = [item['text'] for item in task['query_set']]
    support_embeddings = encode_texts(support_texts)
    query_embeddings = encode_texts(query_texts)
    similarities = cosine_similarity(query_embeddings, support_embeddings)
    max_similarities = similarities.max(axis=1)
    labels = [1 if sim >= threshold else 0 for sim in max_similarities]
    return labels

def process_tasks_and_write_output(input_file_path, output_file_path):
    """处理所有任务并将结果写入CSV文件"""
    with open(output_file_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['task_id', 'record_id', 'label'])  # 写入表头
        task_count = 0  # 用于计数和打印进度
        
        for task in read_data(input_file_path):
            labels = find_positive_samples(task)
            for record_id, label in enumerate(labels):
                writer.writerow([task['task_id'], record_id, label])
            task_count += 1
            if task_count % 2 == 0:  # 每处理十条数据打印一次
                print(f"Processed {task_count} tasks.")

# 设置文件路径
input_file_path = './data/train_data.txt'  # 修改为实际文件路径
output_file_path = './submission.csv'  # 您希望生成的提交文件路径

# 处理数据并输出结果
process_tasks_and_write_output(input_file_path, output_file_path)


Processed 2 tasks.
Processed 4 tasks.


KeyboardInterrupt: 